In [23]:
import pandas as pd
from chronos import ChronosPipeline
import torch
import numpy as np
from utils import preprocess
from mlforecast import MLForecast
from tqdm import tqdm

In [24]:
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-base",
    device_map="cpu",
    torch_dtype=torch.float32, 
)

In [25]:
df = pd.read_csv('data/01_input_history.csv')
df.head()
df_train_null, df_train_inactive, df_train_active, df_validation = preprocess.preprocess_ex1(df)

In [26]:
df_validation['Forecast'] = 0

In [27]:
FORECAST_HORIZON = 12

In [ ]:
for id in tqdm(df_train_active['unique_id'].unique()):
    context = torch.tensor(df_train_active[df_train_active['unique_id'] == id]['Quantity'].values)
    forecast= pipeline.predict(context, FORECAST_HORIZON,num_samples=20)
    median = np.quantile(forecast[0].numpy(), 0.5, axis=0).astype(int)
    df_validation.loc[df_validation['unique_id'] == id, 'Forecast'] = median

100%|██████████| 405/405 [09:01<00:00,  1.34s/it]


In [50]:
def restore_original_format(date_column):
    return date_column.dt.strftime('%b%Y')


In [51]:
df_validation['Month'] = restore_original_format(df_validation['ds'])
df_validation

,unique_id,ds,Quantity,Country,Product,Forecast,Month
0,Japan_MorningMint,2023-01-01,0,Japan,MorningMint,0,Jan2023
1,Japan_MorningMint,2023-02-01,0,Japan,MorningMint,0,Feb2023
2,Japan_MorningMint,2023-03-01,0,Japan,MorningMint,0,Mar2023
3,Japan_MorningMint,2023-04-01,0,Japan,MorningMint,0,Apr2023
4,Japan_MorningMint,2023-05-01,0,Japan,MorningMint,0,May2023
...,...,...,...,...,...,...,...
11995,Russia_HydratingHoney Pet Wipes,2023-08-01,0,Russia,HydratingHoney Pet Wipes,0,Aug2023
11996,Russia_HydratingHoney Pet Wipes,2023-09-01,0,Russia,HydratingHoney Pet Wipes,0,Sep2023
11997,Russia_HydratingHoney Pet Wipes,2023-10-01,0,Russia,HydratingHoney Pet Wipes,0,Oct2023
11998,Russia_HydratingHoney Pet Wipes,2023-11-01,0,Russia,HydratingHoney Pet Wipes,0,Nov2023


In [52]:
df_validation[['Quantity','Country','Product','Month']].to_csv('submissions/validation_chronos.csv', index=False)

In [53]:
df_forecast = df_validation[['Forecast','Country','Product','Month']].copy()
df_forecast.rename(columns={'Forecast': 'Quantity'}, inplace=True)
df_forecast[['Quantity','Country','Product','Month']].to_csv('submissions/submission_chronos.csv', index=False)

In [ ]:
# def restore_original_format(date_column):
#     return date_column.dt.strftime('%b%Y')

# def submission_formatter(df):
#     restored_df = df.copy()
#     restored_df['Country'] = df['unique_id'].str.split('_').str[0]
#     restored_df['Product'] = df['unique_id'].str.split('_').str[1]
#     restored_df['Month'] = restore_original_format(restored_df['ds'])
#     restored_df.drop(columns=['unique_id', 'ds'], inplace=True)
#     return restored_df

    
# df_forecast = submission_formatter(df_forecast)
# df_validation = submission_formatter(df_validation)